In [1]:
# !pip install pyvista[jupyter]

In [2]:
import numpy as np
import pyvista as pv
from bruges import attribute

from scipy.ndimage import label, center_of_mass
import json

import matplotlib.pyplot as plt

pv.set_jupyter_backend("trame")

/home/ab/apps/seismic-anomalies-krr-assistant/.venv/lib/python3.12/site-packages/bruges/__init__.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, DistributionNotFound


In [3]:
CUBE_PATH = "../data/processed/F3_seismic.npy"

In [4]:
import numpy as np
from scipy.ndimage import gaussian_filter


def compute_curvature(seismic, sigma=1.0):
    """
    Вычисляет кривизну как вторую производную по времени.
    """
    # Сглаживание для уменьшения шума
    smoothed = gaussian_filter(seismic, sigma=sigma)

    # Вторая производная по последней оси (время)
    curvature = np.gradient(np.gradient(smoothed, axis=2), axis=2)

    return curvature

# 1. Выявление аномалий в сейсмическом кубе

In [5]:
# --------------------------------------------------
# 1. ЗАГРУЗКА
# --------------------------------------------------
cube = np.load(CUBE_PATH)
print("Полный куб:", cube.shape)

# Фрагмент
subcube = cube[200:400, 300:600, 250:350]
print("Фрагмент:", subcube.shape)

# Нормализация
subcube_norm = (subcube - np.mean(subcube)) / (2 * np.std(subcube))
subcube_norm = np.clip(subcube_norm, -2.0, 2.0)

# --------------------------------------------------
# 2. АТРИБУТЫ
# --------------------------------------------------
dt = 0.004
envelope = np.abs(attribute.envelope(subcube_norm))
inst_freq = attribute.instantaneous_frequency(subcube_norm, dt=dt)
inst_freq = np.nan_to_num(inst_freq, nan=np.nanmean(inst_freq))
curv = compute_curvature(subcube_norm, sigma=0.5)

# --------------------------------------------------
# 3. ВЫРАВНИВАНИЕ ПО ВСЕМ ОСЯМ
# --------------------------------------------------
print("До выравнивания:")
print("  subcube_norm:", subcube_norm.shape)
print("  envelope:", envelope.shape)
print("  inst_freq:", inst_freq.shape)

min_shape = np.minimum.reduce(
    [subcube_norm.shape, envelope.shape, inst_freq.shape]
)

subcube_norm = subcube_norm[: min_shape[0], : min_shape[1], : min_shape[2]]
envelope = envelope[: min_shape[0], : min_shape[1], : min_shape[2]]
inst_freq = inst_freq[: min_shape[0], : min_shape[1], : min_shape[2]]
curv = curv[: min_shape[0], : min_shape[1], : min_shape[2]]

print("После выравнивания:", subcube_norm.shape)

# --------------------------------------------------
# 4. АНОМАЛИИ
# --------------------------------------------------
threshold_amp = np.mean(envelope) + 2.0 * np.std(envelope)
anomaly_mask = envelope > threshold_amp
labeled, n_features = label(anomaly_mask)


# --------------------------------------------------
# 5. ФУНКЦИИ И ЦИКЛ
# --------------------------------------------------
def check_attenuation(env_cube, mask, center_t, depth_window=15):
    t_int = int(center_t)
    if t_int + depth_window >= env_cube.shape[2]:
        return False

    # Амплитуда в аномалии
    amp_in = np.mean(env_cube[mask])

    # Берём НЕПОСРЕДСТВЕННО под аномалией (5-10 отсчётов)
    below_start = t_int + 2
    below_end = min(t_int + 8, env_cube.shape[2])
    if below_end <= below_start:
        return False

    # Только под аномалией (не весь куб!)
    below_mask = mask[:, :, t_int]  # маска на уровне аномалии
    below_region = env_cube[:, :, below_start:below_end]

    # Проектируем маску вниз
    projected_mask = np.tile(
        below_mask[:, :, np.newaxis], (1, 1, below_end - below_start)
    )

    if not np.any(projected_mask):
        return False

    amp_below = np.mean(below_region[projected_mask])

    # Требуем >50% затухания. Локальное, сильное затухание — как у газа.
    return amp_below < (amp_in * 0.5)


anomalies = []
anomalies_ids = {}
for i in range(1, n_features + 1):
    mask_i = labeled == i
    if np.sum(mask_i) < 100:
        continue
    center = center_of_mass(mask_i)
    il, xl, t = center
    mean_amp = float(np.mean(envelope[mask_i]))
    mean_freq = float(np.mean(inst_freq[mask_i]))
    attenuation = check_attenuation(envelope, mask_i, t)
    t_global_index = int(t) + 250
    t_global_ms = t_global_index * 4
    strat_unit = (
        "Sandstone"
        if 1100 <= t_global_ms <= 1300
        else ("Chalk" if t_global_ms < 1000 else "Shale")
    )

    mean_curv = np.mean(curv[mask_i])

    a = {
        "id": f"anomaly_{i:03d}",
        "hasHighAmplitude": mean_amp > 0.8,
        "hasLowFrequency": mean_freq < 50.0,
        "hasInline": int(il) + 200,
        "hasCrossline": int(xl) + 300,
        "hasTimeIndex": t_global_index,
        "hasTimeMs": t_global_ms,
        "exhibitsAttenuationBelow": bool(attenuation),
        "locatedInStratigraphicUnit": strat_unit,
        "isAnticlinal": bool(
            mean_curv > 0
        ),  # антиклиналь = положительная кривизна
    }
    anomalies.append(a)
    anomalies_ids[a["id"]] = a

print(f"\nИзвлечено {len(anomalies)} аномалий.")
if anomalies:
    print(json.dumps(anomalies[0], indent=2))

with open("anomalies_features.json", "w") as f:
    json.dump(anomalies, f, indent=2)

Полный куб: (651, 951, 462)
Фрагмент: (200, 300, 100)
До выравнивания:
  subcube_norm: (200, 300, 100)
  envelope: (200, 300, 100)
  inst_freq: (199, 300, 100)
После выравнивания: (199, 300, 100)

Извлечено 124 аномалий.
{
  "id": "anomaly_002",
  "hasHighAmplitude": true,
  "hasLowFrequency": true,
  "hasInline": 275,
  "hasCrossline": 358,
  "hasTimeIndex": 259,
  "hasTimeMs": 1036,
  "exhibitsAttenuationBelow": false,
  "locatedInStratigraphicUnit": "Shale",
  "isAnticlinal": false
}


In [6]:
print(
    "Всего аномалий: ",
    sum([1 for a in anomalies if a["exhibitsAttenuationBelow"]]),
)

Всего аномалий:  36


# Визуализация куба

In [7]:
# # --------------------------------------------------
# # 3. ПОДГОТОВКА СЕТОК
# # --------------------------------------------------
# grid = pv.ImageData()
# grid.dimensions = np.array(subcube_norm.shape) + 1
# grid.cell_data["Amplitude"] = subcube_norm.flatten(order="F")

# grid_anomaly = pv.ImageData()
# grid_anomaly.dimensions = np.array(anomaly_mask.shape) + 1
# grid_anomaly.cell_data["Anomaly"] = anomaly_mask.astype(np.float32).flatten(
#     order="F"
# )
# contours = grid_anomaly.cell_data_to_point_data().contour([0.5])

# # --------------------------------------------------
# # 4. ВИЗУАЛИЗАЦИЯ: СИЛЬНО ПРОЗРАЧНЫЙ КУБ
# # --------------------------------------------------
# plotter = pv.Plotter(window_size=[1200, 800])

# # 🔹 ОЧЕНЬ ПРОЗРАЧНЫЙ КУБ
# plotter.add_volume(
#     grid,
#     scalars="Amplitude",
#     cmap="seismic",
#     opacity="sigmoid_8",
#     # opacity=[0.0, 0.05, 0.1, 0.2],
#     # opacity=[0.0, 0.0, 0.0, 0.0],
#     blending="composite",
# )

# # 🔸 ЯРКАЯ АНОМАЛИЯ
# if contours.n_points > 0:
#     plotter.add_mesh(
#         contours,
#         color="yellow",  # стандарт для bright spots
#         opacity=0.9,  # почти непрозрачная
#         smooth_shading=True,
#         label="Bright Spot",
#     )

# # Оформление
# plotter.show_axes()
# plotter.add_title("Сейсмический куб", font_size=12)
# plotter.camera_position = "iso"

# plotter.show()

In [8]:
print("Число аномальных вокселей:", anomaly_mask.sum())
if anomaly_mask.sum() == 0:
    print("⚠️ Нет аномалий! Попробуйте уменьшить порог.")

Число аномальных вокселей: 306579


In [9]:
# # Пример: срез по времени, где аномалия максимальна
# time_idx = np.argmax(envelope.mean(axis=(0, 1)))
# plt.figure(figsize=(10, 6))
# plt.imshow(subcube_norm[:, :, time_idx], cmap="gray")
# plt.contour(anomaly_mask[:, :, time_idx], colors="red", linewidths=1)
# plt.title(f"Time slice {time_idx} with anomaly (red)")
# plt.show()

# Генерация OWL-индивидов из JSON

In [10]:
print("Число аномалий до запуска ризонера:", len(anomalies))

Число аномалий до запуска ризонера: 124


In [11]:
INITIAL_ONTOLOGY_PATH = "../ontology/seismic_ontology.owx"
RESULT_INITIAL_ONTOLOGY_PATH = "../ontology/seismic_with_anomalies.owx"
INFERED_ONTOLOGY_PATH = "../ontology/infered_seismic_ontology.owx"

In [12]:
from owlready2 import *
import json

onto = get_ontology(INITIAL_ONTOLOGY_PATH).load()

with onto:
    with open("anomalies_features.json") as f:
        anomalies = json.load(f)

    for a in anomalies:
        ind = onto.BrightSpot(a["id"])

        ind.hasHighAmplitude = a["hasHighAmplitude"]
        ind.hasLowFrequency = a["hasLowFrequency"]
        ind.hasInline = a["hasInline"]
        ind.hasCrossline = a["hasCrossline"]
        ind.hasTimeIndex = a["hasTimeIndex"]
        ind.exhibitsAttenuationBelow = a["exhibitsAttenuationBelow"]
        ind.isAnticlinal = a["isAnticlinal"]

        strat_map = {
            "Sandstone": onto.sandstone_unit,
            "Shale": onto.shale_unit,
            "Chalk": onto.chalk_unit,
        }
        ind.locatedInStratigraphicUnit = [
            strat_map[a["locatedInStratigraphicUnit"]]
        ]

onto.save(RESULT_INITIAL_ONTOLOGY_PATH)
print("Готово!")

Готово!


### Примечание: В Protege выполнить:
- открыть "seismic_with_anomalies.owx", запустить HermiT
- выполнить экспорт infered ontology в INFERED_ONTOLOGY_PATH

In [13]:
# Загрузитка онтологии С ВЫВЕДЕННЫМИ АКСИОМАМИ
onto = get_ontology(INFERED_ONTOLOGY_PATH).load()

candidates = [cls for cls in onto.classes() if "Strong" in cls.name]
print("Найденные классы:", [c.name for c in candidates])

if candidates:
    StrongGasIndicator = candidates[0]
    strong_indicators = list(StrongGasIndicator.instances())
    print(f"Найдено {len(strong_indicators)} индивидов")

    # Список ID StrongGasIndicator (из OWLready2)
    strong_ids = [i.name for i in StrongGasIndicator.instances()]
    print(strong_ids)

else:
    print("Класс StrongGasIndicator не найден!")

Найденные классы: ['StrongGasIndicator']
Найдено 7 индивидов
['anomaly_2023', 'anomaly_2371', 'anomaly_3001', 'anomaly_5105', 'anomaly_5214', 'anomaly_6919', 'anomaly_7712']


In [14]:
coords = [
    [
        anomalies_ids[sg]["hasInline"] - 200,
        anomalies_ids[sg]["hasCrossline"] - 300,
        anomalies_ids[sg]["hasTimeIndex"] - 250,
    ]
    for sg in strong_ids
]

coords = np.array(coords)  # форма: (N, 3)
print("Координаты аномалий:", coords.shape)

Координаты аномалий: (7, 3)


In [15]:
coords

array([[ 41, 108,  42],
       [ 47, 199,  28],
       [ 69, 140,  35],
       [ 96,  89,  65],
       [102, 224,  45],
       [149, 112,  72],
       [169, 210,  59]])

In [16]:
with open(INFERED_ONTOLOGY_PATH) as f:
    count = f.read().count("GasChargedReservoir")
print("Число аномалий после ризонинга:", count)

Число аномалий после ризонинга: 29


In [17]:
with open(INFERED_ONTOLOGY_PATH) as f:
    count = f.read().count("StrongGasIndicator")
print("Число аномалий StrongGasIndicator после ризонинга:", count)

Число аномалий StrongGasIndicator после ризонинга: 9


In [18]:
with open(INFERED_ONTOLOGY_PATH) as f:
    count = f.read().count("WeakGasIndicator")
print("Число аномалий WeakGasIndicator после ризонинга:", count)

Число аномалий WeakGasIndicator после ризонинга: 97


# Визуализация куба и аномалий

In [24]:
# --------------------------------------------------
# 3. ПОДГОТОВКА СЕТОК
# --------------------------------------------------
grid = pv.ImageData()
grid.dimensions = np.array(subcube_norm.shape) + 1
grid.cell_data["Amplitude"] = subcube_norm.flatten(order="F")

grid_anomaly = pv.ImageData()
grid_anomaly.dimensions = np.array(anomaly_mask.shape) + 1
grid_anomaly.cell_data["Anomaly"] = anomaly_mask.astype(np.float32).flatten(
    order="F"
)
contours = grid_anomaly.cell_data_to_point_data().contour([0.5])

# --------------------------------------------------
# 4. ВИЗУАЛИЗАЦИЯ: СИЛЬНО ПРОЗРАЧНЫЙ КУБ
# --------------------------------------------------
plotter = pv.Plotter(window_size=[1200, 800])

# 🔹 ОЧЕНЬ ПРОЗРАЧНЫЙ КУБ
plotter.add_volume(
    grid,
    scalars="Amplitude",
    # cmap="seismic",
    cmap="grey",
    # opacity="sigmoid_8",
    # opacity="linear",
    opacity=[0.0, 0.05, 0.05, 0.05],
    # opacity=[0.0, 0.0, 0.0, 0.0],
    blending="composite",
)

# 🔸 ЯРКАЯ АНОМАЛИЯ
if contours.n_points > 0:
    plotter.add_mesh(
        contours,
        color="yellow",  # стандарт для bright spots
        opacity=0.7,  # почти непрозрачная
        smooth_shading=True,
        label="Bright Spot",
    )

# Создайте облако точек для аномалий
if len(coords) > 0:
    points = pv.PolyData(coords)
else:
    points = None
if points is not None:
    plotter.add_mesh(
        points,
        color="yellow",
        point_size=25,
        render_points_as_spheres=True,
        lighting=True,
    )


# Оформление
plotter.show_axes()
plotter.add_title("Сейсмический куб", font_size=12)
plotter.camera_position = "iso"

plotter.show()

/home/ab/apps/seismic-anomalies-krr-assistant/.venv/lib/python3.12/site-packages/pyvista/core/utilities/points.py:79: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(


Widget(value='<iframe src="http://localhost:43233/index.html?ui=P_0x72f923506a20_4&reconnect=auto" class="pyvi…

In [25]:
plotter.screenshot("seismic_3d.png", scale=2)
# масштаб 2x для высокого DPI